In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import time
import os
from numpy import linalg as la

## set options 
pd.set_option('display.float_format', '{:.2g}'.format) # 항상 float 형식으로
## set parameters
#ROOT = 'PATH/TO/data/processed/'
ROOT = '/home/kddlab/swyoo/data/'
PATH_TO_TRAIN = ROOT + 'train.csv'
PATH_TO_TEST = ROOT + 'test.csv'
PATH_TO_ITEM = ROOT + 'item_metadata.csv'
checkpoint_dir = './checkpoint'
if not os.path.exists(checkpoint_dir): os.mkdir(checkpoint_dir)

In [2]:
## load data
train = pd.read_csv(PATH_TO_TRAIN)
test = pd.read_csv(PATH_TO_TEST)
item = pd.read_csv(PATH_TO_ITEM)


# data exploration
https://datascienceschool.net/view-notebook/7002e92653434bc88c8c026c3449d27b/


In [3]:
print(train.info())
print("===============================")
for col in train.columns:
    print(col,": ",len(train[col].unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15932992 entries, 0 to 15932991
Data columns (total 12 columns):
user_id            object
session_id         object
timestamp          int64
step               int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtypes: int64(2), object(10)
memory usage: 1.4+ GB
None
user_id :  730803
session_id :  910683
timestamp :  518048
step :  3522
action_type :  10
reference :  400277
platform :  55
city :  34752
device :  3
current_filters :  61981
impressions :  1059892
prices :  1066776


In [4]:
train.describe()

,timestamp,step
count,1.6e+07,1.6e+07
mean,1.5e+09,76
std,1.5e+05,1.4e+02
min,1.5e+09,1
25%,1.5e+09,8
50%,1.5e+09,28
75%,1.5e+09,81
max,1.5e+09,3.5e+03


In [5]:
for action in train['action_type'].unique():
    display(train.loc[train.action_type == action][:1])

,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
0,00RL8Z82B2Z1,aff3928535f48,1541037460,1,search for poi,Newtown,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
1,00RL8Z82B2Z1,aff3928535f48,1541037522,2,interaction item image,666856,AU,"Sydney, Australia",mobile,NaN,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
13,00RL8Z82B2Z1,aff3928535f48,1541037543,14,clickout item,109038,AU,"Sydney, Australia",mobile,NaN,3400638|1253714|3367857|5100540|1088584|666916...,95|66|501|112|95|100|101|72|82|56|56|143|70|25...


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
16,02HGRBA06ODU,fa3a53156ae5c,1541030832,1,interaction item info,3377332,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
62,02HGRBA06ODU,fa3a53156ae5c,1541031044,47,interaction item deals,3066244,BR,"Ubatuba, Brazil",mobile,NaN,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
116,03K8AXBL4BX2,ec139e10b9238,1541100038,1,search for destination,"London, United Kingdom",UK,"London, United Kingdom",desktop,NaN,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
125,03P4VFKK12UO,325fafb5fa450,1541107336,3,filter selection,Very Good Rating,US,"Bakersfield, USA",desktop,Very Good Rating,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
178,0473FZ8UNXRS,bcc452f3350eb,1541062453,1,interaction item rating,749441,AU,"Seminyak, Indonesia",desktop,NaN,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
179,0473FZ8UNXRS,bcc452f3350eb,1541062460,2,search for item,1258184,AU,"Legian, Indonesia",desktop,NaN,NaN,NaN


,user_id,session_id,timestamp,step,action_type,reference,platform,city,device,current_filters,impressions,prices
185,06SZHKMYOOI8,4c6062d7cefe4,1541104536,1,change of sort order,interaction sort button,IT,"Bolzano, Italy",mobile,NaN,NaN,NaN


In [6]:
test.info()
print("===============================")
for col in test.columns:
    print(col,": ",len(test[col].unique()))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3782335 entries, 0 to 3782334
Data columns (total 12 columns):
user_id            object
session_id         object
timestamp          int64
step               int64
action_type        object
reference          object
platform           object
city               object
device             object
current_filters    object
impressions        object
prices             object
dtypes: int64(2), object(10)
memory usage: 346.3+ MB
user_id :  250852
session_id :  291381
timestamp :  172502
step :  2815
action_type :  10
reference :  184879
platform :  55
city :  22382
device :  3
current_filters :  21956
impressions :  352789
prices :  355102


In [7]:
test.describe()

,timestamp,step
count,3.8e+06,3.8e+06
mean,1.5e+09,54
std,4.9e+04,1.1e+02
min,1.5e+09,1
25%,1.5e+09,5
50%,1.5e+09,19
75%,1.5e+09,57
max,1.5e+09,2.8e+03


In [8]:
item.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 927142 entries, 0 to 927141
Data columns (total 2 columns):
item_id       927142 non-null int64
properties    927142 non-null object
dtypes: int64(1), object(1)
memory usage: 14.1+ MB


In [9]:
item.describe()

,item_id
count,9.3e+05
mean,5.3e+06
std,3.5e+06
min,5e+03
25%,2.1e+06
50%,5.2e+06
75%,8.7e+06
max,1.1e+07


In [10]:
item[:2]

,item_id,properties
0,5101,Satellite TV|Golf Course|Airport Shuttle|Cosme...
1,5416,Satellite TV|Cosmetic Mirror|Safe (Hotel)|Tele...


In [11]:
platforms = train.platform.unique()
n_platforms = len(platforms)
platformsIdx = pd.Series(data=np.arange(n_platforms), index=platforms).to_dict()
platformsIdx_inv = dict([(v, k) for k, v in platformsIdx.items()])

In [12]:
cities = train.city.unique()
n_cities = len(cities)
citiesIdx = pd.Series(data=np.arange(n_cities), index=cities).to_dict()
citiesIdx_inv = dict([(v, k) for k, v in citiesIdx.items()])

In [13]:
users = train.user_id.unique()
n_users = len(users)
usersIdx = pd.Series(data=np.arange(n_users), index=users).to_dict()
usersIdx_inv = dict([(v, k) for k, v in usersIdx.items()])

In [14]:
A = train[train.columns[[0,6,7,8]]]
# A = train[train.columns]
# A = pd.DataFrame()
A[:1]

,user_id,platform,city,device
0,00RL8Z82B2Z1,AU,"Sydney, Australia",mobile


In [15]:
# T = train.loc[train.action_type == "search for poi"]

In [16]:
# T[:2]

In [17]:
# len(item.item_id.unique())

In [18]:
# items = item.item_id.unique()
# n_items = len(items)
# itemsIdx = pd.Series(data=np.arange(n_items), index=items).to_dict()
# itemsIdx_inv = dict([(v, k) for k, v in itemsIdx.items()])

In [19]:
# # Get one hot encoding of columns B
# one_hot = pd.get_dummies(T['reference'])
# # # Drop column B as it is now encoded
# # A = A.drop('device',axis = 1)
# # # Join the encoded df
# # A = A.join(one_hot)
# one_hot.shape

In [20]:
# A[:2]

In [21]:
# one_hot[:2]

In [24]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(train['platform'])
A = A.drop('platform',axis = 1)
A = A.join(one_hot)
A[:2]

,user_id,city,device,AA,AE,AR,AT,AU,BE,BG,...,SI,SK,TH,TR,TW,UK,US,UY,VN,ZA
0,00RL8Z82B2Z1,"Sydney, Australia",mobile,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,00RL8Z82B2Z1,"Sydney, Australia",mobile,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
# Get one hot encoding of columns B
one_hot = pd.get_dummies(train['device'])
A = A.drop('device',axis = 1)
A = A.join(one_hot)
A[:2]

,user_id,city,AA,AE,AR,AT,AU,BE,BG,BR,...,TR,TW,UK,US,UY,VN,ZA,desktop,mobile,tablet
0,00RL8Z82B2Z1,"Sydney, Australia",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,00RL8Z82B2Z1,"Sydney, Australia",0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [28]:
B = A[:10000].groupby('user_id').sum()

In [29]:
B

,AA,AE,AR,AT,AU,BE,BG,BR,CA,CH,...,TR,TW,UK,US,UY,VN,ZA,desktop,mobile,tablet
user_id,,,,,,,,,,,,,,,,,,,,,
00RL8Z82B2Z1,0,0,0,0,16,0,0,0,0,0,...,0,0,0,0,0,0,0,0,16,0
02HGRBA06ODU,0,0,0,0,0,0,0,65,0,0,...,0,0,0,0,0,0,0,0,65,0
02SRUT1NQYH1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35,0
03K8AXBL4BX2,0,0,0,0,0,0,0,0,0,0,...,0,0,7,0,0,0,0,7,0,0
03P4VFKK12UO,0,0,0,0,0,0,0,0,0,0,...,0,0,0,55,0,0,0,55,0,0
0473FZ8UNXRS,0,0,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,0
066TUPQWUEV5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,3,0
06S61EKCW1JY,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
06SZHKMYOOI8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,11,0


In [30]:
B.values.shape

(574, 58)

In [31]:
M = B.values


In [32]:
M = M/la.norm(M, 'fro')

In [33]:
print(M)

[[0.         0.         0.         ... 0.         0.01151625 0.        ]
 [0.         0.         0.         ... 0.         0.04678478 0.        ]
 [0.         0.         0.         ... 0.         0.0251918  0.        ]
 ...
 [0.         0.         0.         ... 0.03382899 0.         0.        ]
 [0.         0.         0.         ... 0.         0.00071977 0.        ]
 [0.         0.         0.         ... 0.         0.01943368 0.        ]]


In [34]:
from sklearn.cluster import KMeans

In [35]:
km = KMeans(n_clusters=10, random_state=0).fit(M)
km

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [40]:
len(test)

3782335

In [48]:
sample = A[:82335].groupby('user_id').sum()
print(sample.values.shape)

(4625, 58)


In [49]:
km.predict(sample)

array([4, 1, 3, ..., 1, 4, 1], dtype=int32)

cluster를 정할 수 있어, 그 안에서 similarity가 높은 target variable 을 예상 할 수있다. 